# Sm GFF HHPred table



## Aim

GFF annotation might be sometimes misleading when it comes to function prediction (especially if generated from simple blast). In order to complement this annotation, we will use a second method based on similarity detection relying on Hidden Markov Model as implement in HHsearch from the [HH-suite](https://doi.org/10.1093/bioinformatics/bti125). We will analyze the predicted protein sequences from the latest version of the genome, sequences corresponding to each isoform (i.e., transcript) predicted from the genome. We will generate a table with both GFF annotation (from the latest GFF file version) and HHsearch annotation.

Historically, this approach was very helpful for our [oxamniquine resistance work](https://doi.org/10.1126/science.1243106) in *Schistosoma mansoni*. The gene involved in drug activation was known to have a sulfotransferase activity from previous studies. Our genetic approach revealed promising candidates which were annotated as epimerases in the v5 genome of *S. mansoni*, but epimerases do not have the expected sulfotransferase activity. However, the candidates turned out to be closer to sulfotransferases than epimerases after using the HHPred server and this was later biochemically confirmed. This convinced us to use this approach to systematically complement GFF annotation.



## Environment and databases

### Conda environment

This allows for a better replication of the analysis and database generation.

In [ ]:
# Check if conda available
[[ ! $(which conda 2> /dev/null) ]] && echo "conda not available in \$PATH. Please interrupt the kernel and fix the situation." && sleep inf

# Creating conda environment
conda env create -f .env/env.yml

This cell must be run each time a new session of Jupyter is run.

In [ ]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate hhpred

# PERL lib (for splitfasta.pl)
export PERL5LIB=$CONDA_PREFIX/lib/5.26.2:$CONDA_PREFIX/scripts/

### Download databases

**NB:** It can take a very long time (up to 6h) to download and decompress the databases (up to 6h).

In [ ]:
db_dir="db"
[[ ! -d "$db_dir" ]] && mkdir -p "$db_dir"

# Download PDB database
wget -P "$db_dir" http://wwwuser.gwdg.de/~compbiol/data/hhsuite/databases/hhsuite_dbs/pdb70_from_mmcif_200902.tar.gz
tar -C "$db_dir" --use-compress-program=pigz -xvf db/pdb70_from_mmcif_200902.tar.gz

# Download Pfam database
wget -P "$db_dir" http://wwwuser.gwdg.de/~compbiol/data/hhsuite/databases/hhsuite_dbs/pfamA_32.0.tar.gz
tar -C "$db_dir" --use-compress-program=pigz -xvf db/pfamA_32.0.tar.gz

# Download SCOP database
wget -P "$db_dir" http://wwwuser.gwdg.de/~compbiol/data/hhsuite/databases/hhsuite_dbs/scop70_1.75_hhsuite3.tar.gz
tar -C "$db_dir" --use-compress-program=pigz -xvf db/scop70_1.75_hhsuite3.tar.gz

# Allow file read by everyone (otherwise hhsearch will emits an open file error)
chmod +r "$db_dir"/*

## Analysis and database creation

### Generate HHPred database

This step downloads the predicted protein sequences from [WomBase ParaSite website](https://parasite.wormbase.org/Schistosoma_mansoni_prjea36577/Info/Index/) (version WBPS14). The sequences are split in individual fasta files. Path to the files is stored into a text file (list) and the list is split in batches of 100 sequences (this can be changed to generate shorter lists in order to speed up the next step).

In [ ]:
# Download the protein sequences
wget "ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.protein.fa.gz"
pigz -d schistosoma_mansoni.PRJEA36577.WBPS14.protein.fa.gz

# Split the protein fasta file in individual files
[[ ! -d data ]] && mkdir data
mv schistosoma_mansoni.PRJEA36577.WBPS14.protein.fa data/
cd data
perl $CONDA_PREFIX/scripts/splitfasta.pl schistosoma_mansoni.PRJEA36577.WBPS14.protein.fa
cd ..
mv data/schistosoma_mansoni.PRJEA36577.WBPS14.protein.fa .

# Create a list file of fasta paths to process and split the list for parallelization
[[ ! -d list.d ]] && mkdir list.d
ls -1 data/* > list
split -l 100 -a 3 -d list list.d/list.

This step uses the custom `hhpred-ann.sh` script to run `hhsearch` on sequences given by a split list and to generate a corresponding list with the best annotation for each sequence. This is efficient only if the split lists are run in parallel on a computer cluster. In the cell below, each split list is submitted to a node using Oracle Grid Engine. This parallelization helps to speed up tremendously the analysis (run takes ~4h instead of ~400h).

The script takes in account all the databases and a minimum probability of 50% to retain the best match as recommended by the [HH-suite documentation](https://github.com/soedinglab/hh-suite/wiki#how-can-i-verify-if-a-database-match-is-homologous). Help on script usage is available using `./hhpred-ann.sh -h`.

In [ ]:
# Status folder to store the jobs' status
[[ ! -d status ]] && mkdir status

# Submit split list to the nodes
for i in list.d/*
do
    qsub -V -cwd -o status -j y -r y -S /bin/bash hhpred-ann.sh -i "$i" -d db/pdb70 db/scop70_1.75 db/pfam -p 50
done

When all the sequences have been annotated, this final step concatenates all the annotations in a single file.

In [ ]:
# Build the database
db_hh=hhpred_ann_$(date +%F).db
cat results/list.* > "$db_hh"

### Extract annotations from the GFF file.

This step downloads the GFF file from WomBase ParaSite website (version WBPS14).

In [ ]:
# Download the GFF
wget ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz
pigz -d schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz

To extract the protein function annotations from the GFF, we use a single command pipeline that executes the following steps:
- Extract the info column
- Extract the ID and annotation
- Clean the information and add NA when needed
- Convert the HTML code in to normal characters

In [ ]:
db_gff=gff_table.tsv

# Extract ID and annotation
awk '$3 == "mRNA" {print $0}' schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3 |\
    cut -f 9 |\
    awk -F ':' '{print $2"\t"$6}' |\
    sed -r "s/^(.*);.*\t/\1\t/g ; s/ %0A.*//g ; s/\t$/\tNA/" |\
    sed 's@+@ @g;s@%@\\x@g' | xargs -P $(nproc) -n1 -d '\n' printf "%b\n" > gff_table.tsv

### Merge GFF and HHPred annotations.

This final step merges both GFF and HHPred annotations in a single file for each isoform (i.e., transcript).

In [ ]:
# Join GFF annotation and HHPred annotation
join -t $'\t' <(sort -k 1 "$db_gff") <(sort -k 1 "$db_hh") > Sm_transcript_table_gff-hhpred_$(date +%F).tsv 

# Add header
sed -i -r "1s/^/#Transcript_ID\tGFF_annotation\tHHPred_annotation\n/" Sm_transcript_table_gff-hhpred_$(date +%F).tsv